# Sensordata fra Garmin løpeklokke

Datasettet `garmindata.csv` har vi lastet ned fra en personlig brukerkonto på connect.garmin.com. Datasettet består av sensormålinger fra 56 løpeturer, f.eks. tid, avstand, høydemeter, stegfreksens og puls. I tillegg regner løpeklokkens programvare ut andre størrelser slik som estimert kaloriforbuk. Vi er hovedsakelig interresert i sammenhengen mellom *tid og distanse*. 

## Læringsmål: Korrelasjon og minste kvadratsums rette linje (regresjonslinje)

### Delmål
1. Laste inn og behandle et komplekst datasett
2. Plotte kryssplott, identifisere og håndtere uteliggere
3. Regne ut korrelasjon og regresjonslinje

*Hva er relevant for tellende vurdering? Erfaringen med å håndtere datasett i Python får du nytte av i den tellende prosjektoppgaven, og i resten av utdanningen din. Du skal ikke skrive kode selv på eksamen, men du må kunne lese, tolke og forklare utskrift og plott fra ulike statistikk-beregninger i Python.*

## Steg 1: Laste inn nødvendige python-pakker

In [ ]:
import pandas as pd # lese data fra csv-fil og bruke DataFrames
import seaborn as sns # plotting
sns.set(style = 'whitegrid', font_scale = 1.5) # utseende av plott
import matplotlib.pyplot as plt # mer plotting
import numpy as np # matematikk
import statsmodels.api as sms # regresjonen
import statsmodels.formula.api as smf # formel for regresjonen

## Steg 2: Lese inn og behandle data

In [ ]:
df = pd.read_csv('https://www.math.ntnu.no/emner/IST100x/python_felles/garmindata.csv')
df.shape # dimenensjonen til datasettet

In [ ]:
print(df.columns) # Alle kolonnenavn

In [ ]:
# Merk at en del kolonnenavn inneholder punktum
# Dette er generelt uheldig når vi programmerer, så vi 
# erstatter alle punktum med underscore

df.columns = df.columns.str.replace("[.]", "_")

In [ ]:
df.head() # ser på de 5 første linjene i datasettet

Merk at kolonnen 'Time' inneholder tid på formatet timer:minutter:sekunder. Dette må vi kode om til desimaltall, og det mest naturlige er å bruke minutter som enhet. 

In [ ]:
# Vi lager oss en funksjon som koder fra timer:minutter:sekunder til minutter

def get_min(time_str):
    h, m, s = time_str.split(':')
    return round(int(h)*60 + int(m) + int(s)/60 , 2)

# Lager en TimeMin kolonne i datasettet
df['TimeMin'] = df['Time'].apply(get_min)

## Steg 2: Kryssplott og kvalitetskontroll

Tabellen vi har lastet inn er for stor til at vi klarer å kvalitetssjekke observasjoner og få et inntrykk av dataene. Siden vi hovedsakelig er interessert i sammenhengen mellom *tid og distanse* plotter vi et kryssplott av disse observasjonene. 

In [ ]:
# Plotter tid mot distanse for å få et intrykk av datasettet

sns.relplot(x='Distance', y='TimeMin',data = df)
plt.xlim(0,10); plt.ylim(0,60)

plt.ylabel('Tid [min]'); plt.xlabel('Distanse [km]')

# Vi identifiserer en "rar" observasjon:
# En løpetur på under 4 km som tok over 30 minutter!
# Kanskje er dette en gåtur som har blit feilregistrert?

In [ ]:
# Vi ser på observasjonene av gjennomsnittshastighet, Avg.Pace
# tips: print(df.columns) kan brukes for å se på alle kolonnetitlene
# Merk: Avg.Pace er skrevet i min:sek per kilometer
# Vi lager oss derfor en ny kolonne med minutter som desimaltall
def get_min2(time_str):
    m, s = time_str.split(':')
    return round(int(m) + int(s)/60 , 2)

df['Hastighet'] = df['Avg_Pace'].apply(get_min2)

# Plotter boksplott over hastighet
sns.boxplot(y = 'Hastighet', data = df)

# Her ser vi en klar uteligger! 
# En hastighet (min/km) på voer 9 minutter per kilometer betyr at dette var en 
# gåtur og hører ikke hjemme i løpe-tur datasettet

In [ ]:
np.argmax(df['Hastighet']) # Hvilken index har uteliggeren?

In [ ]:
# Fjerner uteligger fra datasettet og lager det samme kryssplottet som før
dfny = df.drop(21)
sns.relplot(x='Distance', y='TimeMin',data = dfny)
plt.xlim(0,10); plt.ylim(0,60)
plt.ylabel('Tid [min]'); plt.xlabel('Distanse [km]')

In [ ]:
dfny.to_csv('garmindata2.csv',index = False)

In [ ]:
dfny.shape

## Steg 3: Korrelasjon og regresjonslinje

In [ ]:
# Regne ut korrelasjon (med Pandas)
round(dfny['Distance'].corr(dfny['TimeMin']),3)

In [ ]:
# Regne ut regresjonslinja (minste kvadratsum) og skrive ut resultatet
regresjon = smf.ols('TimeMin~Distance', data=dfny).fit()
regresjon.params

In [ ]:
# Plotte observasjoner og regresjonslinje
sns.lmplot(x = 'Distance', y = 'TimeMin', data = dfny, ci = None)
plt.xlim(0,10); plt.ylim(0,60)  
plt.ylabel('Tid [min]'); plt.xlabel('Distanse [km]')
plt.savefig('plot4.png',bbox_inches='tight') 

## Bonus: illustrasjon av datasett med lavere korrelasjon

In [ ]:
sns.relplot(x='Elev_Gain', y='Calories',data = dfny, color = "darkgreen")